In [1]:
import numpy as np 
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
df = pd.read_csv('dataset/fer2013.csv')
print(df.shape)
df.head()

(35887, 3)


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
df.emotion.unique()

array([0, 2, 4, 6, 3, 5, 1], dtype=int64)

In [5]:
INTERESTED_LABELS = [3, 4, 6, 0, 2]

In [6]:
df = df[df.emotion.isin(INTERESTED_LABELS)]
df.shape

(31338, 3)

In [7]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)

In [8]:
img_array.shape

(31338, 48, 48, 1)

In [9]:
le = LabelEncoder()
img_labels = le.fit_transform(df.emotion)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

(31338, 5)

In [10]:
img_labels

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.1, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((28204, 48, 48, 1), (3134, 48, 48, 1), (28204, 5), (3134, 5))

In [12]:
X_train = X_train / 255.
X_valid = X_valid / 255.

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [14]:
model = Sequential()

In [15]:
model.add(Conv2D(200,(3,3),input_shape=(48,48,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(200,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(100,activation='relu'))
model.add(Dense(5,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
cp = ModelCheckpoint('model-best',monitor='val_accuracy',verbose=0,save_best_only=True)
model.fit(X_train,y_train,batch_size=32,epochs=15,callbacks=[cp],validation_split=0.20)

Train on 22563 samples, validate on 5641 samples
Epoch 1/15
22563/22563 [==============================] - 625s 28ms/step - loss: 1.5392 - accuracy: 0.3231 - val_loss: 1.4871 - val_accuracy: 0.3611
Epoch 2/15
22563/22563 [==============================] - 610s 27ms/step - loss: 1.4268 - accuracy: 0.4029 - val_loss: 1.3741 - val_accuracy: 0.4260
Epoch 3/15
22563/22563 [==============================] - 602s 27ms/step - loss: 1.3579 - accuracy: 0.4381 - val_loss: 1.3079 - val_accuracy: 0.4565
Epoch 4/15
22563/22563 [==============================] - 599s 27ms/step - loss: 1.3102 - accuracy: 0.4627 - val_loss: 1.3168 - val_accuracy: 0.4648
Epoch 5/15
22563/22563 [==============================] - 599s 27ms/step - loss: 1.2783 - accuracy: 0.4739 - val_loss: 1.2581 - val_accuracy: 0.4845
Epoch 6/15
22563/22563 [==============================] - 601s 27ms/step - loss: 1.2463 - accuracy: 0.4901 - val_loss: 1.2359 - val_accuracy: 0.4930
Epoch 7/15
22563/22563 [==============================] -

In [17]:
from keras.models import load_model

In [18]:
model = load_model('model-best')

In [23]:
import cv2
face_detect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [35]:
source = cv2.VideoCapture(0)

while 1:
    not_to_use, image = source.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detect.detectMultiScale(gray, 1.3,5)
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+w, x:x+w]
        resized_face = cv2.resize(face_roi, (48, 48))
        normalized_Face = resized_face/255
        reshaped_face = np.reshape(normalized_Face, (1, 48, 48, 1))
        result = model.predict(reshaped_face)[0]
        print("Probabilities : ",result)
        maximum = max(result)
        index = np.where(result==maximum)
        if index[0] == 0:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, "angry", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        if index[0] == 1:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, "fear", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
        if index[0] == 2:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, "happy", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
        if index[0] == 3:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, "sad", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
        if index[0] == 4:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, "neutral", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
    cv2.imshow('Verzeo Project', image)
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cv2.destroyAllWindows()
source.release()

Probabilities :  [0.40351257 0.11155055 0.18670905 0.22301294 0.07521488]
Probabilities :  [0.4122199  0.09366846 0.21099916 0.2057814  0.07733113]
Probabilities :  [0.41038042 0.10831936 0.19108528 0.21335772 0.0768573 ]
Probabilities :  [0.29870915 0.12579414 0.08051956 0.3873598  0.10761743]
Probabilities :  [0.28833982 0.13816756 0.08108752 0.38608748 0.10631763]
Probabilities :  [0.27172405 0.10636229 0.10635106 0.41341445 0.10214823]
Probabilities :  [0.20342493 0.11066858 0.02010725 0.5546143  0.11118488]
Probabilities :  [0.19050306 0.1177545  0.02089259 0.55962366 0.11122613]
Probabilities :  [0.20875356 0.09132891 0.03012401 0.52979773 0.13999574]
Probabilities :  [0.25746617 0.13273008 0.00463604 0.42273912 0.18242861]
Probabilities :  [0.19847696 0.16745675 0.00508005 0.47530192 0.15368429]
Probabilities :  [0.15290947 0.15967631 0.00502241 0.50613266 0.1762591 ]
Probabilities :  [0.20848916 0.15405259 0.0026902  0.30667588 0.3280921 ]
Probabilities :  [0.17092533 0.1585454

Probabilities :  [0.20713925 0.13159128 0.00828535 0.13861494 0.5143692 ]
Probabilities :  [0.19218643 0.13229609 0.00661426 0.1311919  0.5377114 ]
Probabilities :  [0.18224607 0.15600157 0.01551464 0.11198206 0.5342557 ]
Probabilities :  [0.17239328 0.12682424 0.02277878 0.10766686 0.5703368 ]
Probabilities :  [0.16081727 0.10913284 0.01555539 0.13910896 0.5753855 ]
Probabilities :  [0.14731136 0.11977299 0.02798561 0.135347   0.56958306]
Probabilities :  [0.06369362 0.07214968 0.13669688 0.09849799 0.62896186]
Probabilities :  [0.05748173 0.07276753 0.21060804 0.09493694 0.56420577]
Probabilities :  [0.06349802 0.04591067 0.31946424 0.07245453 0.49867257]
Probabilities :  [0.0418888  0.02919523 0.3768778  0.06481518 0.48722306]
Probabilities :  [0.03065152 0.01893417 0.5865489  0.04499998 0.3188654 ]
Probabilities :  [0.03554914 0.03402011 0.5321417  0.06992508 0.328364  ]
Probabilities :  [0.02540527 0.01040087 0.6683151  0.02032137 0.27555734]
Probabilities :  [0.03547614 0.0150056

Probabilities :  [8.6545362e-04 8.8581175e-04 9.8602700e-01 3.5328530e-03 8.6887823e-03]
Probabilities :  [0.00205297 0.00109573 0.98151153 0.00525735 0.01008249]
Probabilities :  [5.3948956e-04 5.1503134e-04 9.9112517e-01 2.2114040e-03 5.6088869e-03]
Probabilities :  [4.3242122e-04 4.5591712e-04 9.9261606e-01 2.0152663e-03 4.4803643e-03]
Probabilities :  [9.2861586e-04 9.6828892e-04 9.8690993e-01 2.8896963e-03 8.3035352e-03]
Probabilities :  [0.00195479 0.00117655 0.9797442  0.00279677 0.01432761]
Probabilities :  [0.00248709 0.00183254 0.9776512  0.00540619 0.01262297]
Probabilities :  [0.00109525 0.00105831 0.9845657  0.00461764 0.00866303]
Probabilities :  [8.1253366e-04 8.8740309e-04 9.8593163e-01 3.6813878e-03 8.6869597e-03]
Probabilities :  [0.00379482 0.00300171 0.96155804 0.01299086 0.01865454]
Probabilities :  [0.00266545 0.00226011 0.9692404  0.00986774 0.01596626]
Probabilities :  [0.00114561 0.001356   0.9807505  0.00520818 0.01153972]
Probabilities :  [0.00272767 0.002322

Probabilities :  [0.05666809 0.08508809 0.09982906 0.10310389 0.6553108 ]
Probabilities :  [0.04845321 0.09325252 0.09891244 0.10949344 0.64988834]
Probabilities :  [0.07989962 0.11155011 0.10536277 0.1439275  0.55925995]
Probabilities :  [0.07510178 0.11755057 0.10404537 0.1439003  0.55940205]
Probabilities :  [0.0534645  0.08042172 0.10491773 0.10507651 0.65611964]
Probabilities :  [0.05171772 0.10113438 0.13860539 0.11015781 0.5983847 ]
Probabilities :  [0.04012971 0.09317825 0.12087984 0.117503   0.6283092 ]
Probabilities :  [0.04826094 0.07966577 0.09422255 0.10186198 0.67598873]
Probabilities :  [0.07668182 0.09505259 0.07893161 0.15801425 0.5913196 ]
Probabilities :  [0.05158725 0.11856022 0.04707988 0.15615484 0.6266178 ]
Probabilities :  [0.07473717 0.11061458 0.0895141  0.14134255 0.5837916 ]
Probabilities :  [0.08933986 0.10428552 0.02892085 0.19480301 0.5826508 ]
Probabilities :  [0.05821007 0.10102677 0.03799092 0.17512488 0.6276474 ]
Probabilities :  [0.1032801  0.0793855

Probabilities :  [0.01297074 0.02953749 0.4802916  0.05401608 0.423184  ]
Probabilities :  [0.03328    0.06694931 0.11643524 0.081042   0.70229346]
Probabilities :  [0.01789727 0.04346934 0.4432332  0.05779579 0.43760452]
Probabilities :  [0.0244043  0.05096088 0.47122878 0.06430061 0.38910553]
Probabilities :  [0.03107132 0.05729713 0.3235151  0.08272774 0.5053887 ]
Probabilities :  [0.03159458 0.06602821 0.3137773  0.09190398 0.49669597]
Probabilities :  [0.01515431 0.03678554 0.42695254 0.06124599 0.45986158]
Probabilities :  [0.0209803  0.03935542 0.5149544  0.05164982 0.37306005]
Probabilities :  [0.01982351 0.03641474 0.50358725 0.052228   0.38794655]
Probabilities :  [0.01904761 0.03240905 0.52898884 0.04671687 0.37283757]
Probabilities :  [0.01086302 0.02914383 0.5200744  0.04300394 0.3969147 ]
Probabilities :  [0.01897887 0.03476469 0.33841118 0.05786185 0.5499834 ]
Probabilities :  [0.01698894 0.03345786 0.47642714 0.04249868 0.43062732]
Probabilities :  [0.01379763 0.0269669

...

Probabilities :  [0.09069499 0.19194818 0.00137909 0.19177952 0.5241982 ]
Probabilities :  [0.0845987  0.19803531 0.00133887 0.18061264 0.5354145 ]
Probabilities :  [0.05671534 0.22852515 0.00058857 0.27885437 0.43531665]
Probabilities :  [0.06388568 0.21906063 0.00075012 0.23543568 0.48086786]
Probabilities :  [0.06067446 0.20475383 0.00083976 0.234634   0.49909797]
Probabilities :  [0.07971068 0.19891296 0.00070508 0.26084393 0.4598273 ]
Probabilities :  [0.07803199 0.20751703 0.00057928 0.2905674  0.4233043 ]
Probabilities :  [0.07479025 0.21841682 0.00067926 0.26428747 0.4418262 ]
Probabilities :  [0.09382617 0.18750411 0.00065178 0.25223807 0.46577987]
Probabilities :  [0.08247226 0.17409801 0.0005377  0.2574244  0.48546752]
Probabilities :  [0.09206056 0.19086275 0.00074095 0.26032233 0.4560134 ]
Probabilities :  [0.08570645 0.20598732 0.00093893 0.2515261  0.45584124]
Probabilities :  [0.08570645 0.20598732 0.00093893 0.2515261  0.45584124]
Probabilities :  [0.08285204 0.2322331

Probabilities :  [0.09310772 0.19260263 0.0006506  0.2596658  0.4539733 ]
Probabilities :  [0.0853406  0.21749647 0.00094959 0.28214    0.41407344]
Probabilities :  [0.07727171 0.21491005 0.00061362 0.27395588 0.43324876]
Probabilities :  [0.07727171 0.21491005 0.00061362 0.27395588 0.43324876]
Probabilities :  [0.08087262 0.25189182 0.00057952 0.29174766 0.37490827]
Probabilities :  [0.08102331 0.22187778 0.00069647 0.2921855  0.40421686]
Probabilities :  [0.08971981 0.2255523  0.00061716 0.29108247 0.39302823]
Probabilities :  [0.0991692  0.18006939 0.000786   0.25532824 0.46464717]
Probabilities :  [0.08643995 0.21171156 0.00076585 0.26297116 0.43811145]
Probabilities :  [0.09702927 0.19386245 0.00075239 0.2553548  0.4530011 ]
Probabilities :  [0.09122483 0.21513687 0.00098359 0.27563897 0.41701573]
Probabilities :  [0.08563401 0.2140012  0.0006482  0.27176547 0.42795104]
Probabilities :  [0.07721425 0.2389998  0.00065898 0.2933371  0.38978982]
Probabilities :  [0.07855321 0.2467451

Probabilities :  [0.08007626 0.21171527 0.00255836 0.20122448 0.5044256 ]
Probabilities :  [0.07487919 0.15749718 0.00123114 0.19450185 0.57189065]
Probabilities :  [0.07184483 0.14834073 0.00131715 0.21397278 0.5645245 ]
Probabilities :  [0.10810355 0.1816431  0.00130034 0.23582502 0.47312793]
Probabilities :  [0.09406784 0.19404869 0.00324623 0.22148213 0.48715514]
Probabilities :  [0.08042437 0.1800443  0.00190875 0.20062055 0.537002  ]
Probabilities :  [0.08089773 0.16403416 0.00232172 0.17529675 0.5774496 ]
Probabilities :  [0.07400092 0.16913734 0.00193105 0.16401456 0.59091616]
Probabilities :  [0.06497245 0.18167605 0.00218805 0.14418872 0.6069747 ]
Probabilities :  [0.06502172 0.16079448 0.00153272 0.14030221 0.63234884]
Probabilities :  [0.11331443 0.23282638 0.00174607 0.16014452 0.4919686 ]
Probabilities :  [0.06742478 0.25290108 0.0018333  0.15062541 0.52721536]
Probabilities :  [0.05691903 0.25730756 0.00178558 0.1377828  0.5462051 ]
Probabilities :  [0.11201181 0.2023736

Probabilities :  [0.07309039 0.26454014 0.00347247 0.15024492 0.5086521 ]
Probabilities :  [0.06226569 0.2509498  0.00383148 0.15218388 0.53076917]
Probabilities :  [0.09153137 0.17303164 0.00998123 0.14378004 0.5816757 ]
Probabilities :  [0.08020279 0.19814885 0.01204734 0.15317917 0.55642194]
Probabilities :  [0.06407872 0.23674391 0.00762019 0.13142245 0.56013477]
Probabilities :  [0.08075909 0.24458274 0.00619403 0.13495928 0.5335049 ]
Probabilities :  [0.08996109 0.27099642 0.00541373 0.14638758 0.48724118]
Probabilities :  [0.08839955 0.27328575 0.00729531 0.13762604 0.49339336]
Probabilities :  [0.10859827 0.270737   0.00632047 0.14856367 0.46578056]
Probabilities :  [0.10885405 0.22434877 0.00390334 0.17234805 0.4905458 ]
Probabilities :  [0.13935126 0.25949377 0.00355626 0.16909948 0.42849922]
Probabilities :  [0.07577624 0.2224781  0.00226661 0.18772677 0.5117523 ]
Probabilities :  [0.08404351 0.206463   0.00216461 0.1978671  0.5094618 ]
Probabilities :  [0.09701917 0.2068250

Probabilities :  [5.5575452e-04 1.3760291e-04 9.9296737e-01 3.3809355e-04 6.0011563e-03]
Probabilities :  [1.1936933e-03 3.5788916e-04 9.8756593e-01 9.2437584e-04 9.9580847e-03]
Probabilities :  [4.1376567e-04 1.3737350e-04 9.9131000e-01 4.2179128e-04 7.7170720e-03]
Probabilities :  [3.3762380e-03 9.3144551e-04 9.6675849e-01 2.5579028e-03 2.6375925e-02]
Probabilities :  [3.3762380e-03 9.3144551e-04 9.6675849e-01 2.5579028e-03 2.6375925e-02]
Probabilities :  [0.00634305 0.00210604 0.9339496  0.00578591 0.05181544]
Probabilities :  [0.02226726 0.01045294 0.67823106 0.02276083 0.26628786]
Probabilities :  [0.03279638 0.0123761  0.67351943 0.02803884 0.2532692 ]
Probabilities :  [0.06849565 0.02873114 0.49706376 0.06855353 0.3371559 ]
Probabilities :  [0.06699004 0.03283295 0.50246847 0.06309424 0.33461434]
Probabilities :  [0.04918901 0.02995913 0.4164714  0.0673901  0.43699035]
Probabilities :  [0.05987763 0.04152848 0.28847462 0.09845228 0.51166695]
Probabilities :  [0.05195073 0.053009